In [1]:
%matplotlib inline
import numpy as np
from PIL import Image
from keras.models import model_from_json
from keras.optimizers import RMSprop 

import json
import csv

Using TensorFlow backend.
/home/rallen/anaconda3/envs/dl/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
notes_model_file   = 'data/results/note_model_180114.json'
notes_weights_file = 'data/results/note_mask_weights_180114_120637.h5'
with open(notes_model_file,"r") as f:
    json_string = f.read()
notes_model = model_from_json(json_string)
notes_model.load_weights(notes_weights_file)

In [3]:
length_model_file   = 'data/results/length_model_180114.json'
length_weights_file = 'data/results/length_mask_weights_180114_120637.h5'
with open(length_model_file,"r") as f:
    json_string = f.read()
length_model = model_from_json(json_string)
length_model.load_weights(length_weights_file)

In [4]:
def get_notes(base_name):
    notes = []
    with open(f'../setup/{base_name}.rmf') as csvfile:
        for row in csv.reader(csvfile):
            if row[0] == 'note':
                notes.append(row[1:])  
    return notes

def get_notes_lengths(note_pred,length_pred):
    assert(note_pred.shape[0] == length_pred.shape[0])
    num_preds = note_pred.shape[0]
    notes_lengths = []
    for i in range(num_preds):
        note_class = note_pred[i].argmax()
        length_class = length_pred[i].argmax()
        note_confidence = note_pred[i,note_class]
        #if note_confidence > 0.99:
        #    note_confidence = "high"
        #elif note_confidence > 0.8:
        #    note_confidence = "med"
        #else:
        #    note_confidence = "low"
        note_index = note_class-2
        note_name = "C C# D D# E F F# G G# A A# B".split()[note_index % 12]
        note_octave = int(note_index/12) - 2
        if note_index == -2:
            note_name = 'X'
            note_octave = 0
        elif note_index == -1:
            note_name = 'R'
            note_octave = 0
        length_confidence = length_pred[i,length_class]
        #if length_confidence > 0.99:
        #    length_confidence = "high"
        #elif length_confidence > 0.8:
        #    length_confidence = "med"
        #else:
        #    length_confidence = "low"
        length = length_class/4
        if note_index == -2 and note_confidence > 0.8:#note_confidence != "low":
            continue # no note
        notes_lengths.append([i,
                              (note_index,note_name,note_octave,note_confidence),
                              (length, length_confidence)])
    return notes_lengths

def refine_notes_lengths(raw_notes_lengths):
    notes_lengths = []
    cur = []
    last_index = -100
    for x in raw_notes_lengths:
        if x[0] == last_index + 1:
            cur.append(x[1:])
        else:
            num_samples = len(cur)
            if num_samples > 0:
                conf = np.zeros(num_samples)
                for i,c in enumerate(cur):
                    conf[i] = c[0][-1]
                mci = conf.argmax()
                notes_lengths.append((cur[mci][0][0],cur[mci][1][0]))
            cur = []
        last_index = x[0]
    return notes_lengths

def get_crop_inputs(cur_score_info):
    img_crop_width = 32
    img_step = 2
    img_crop_height = int(cur_score_info['height'])
    img_count = cur_score_info['width']//img_step
    cur_img = Image.open('data/train2/' + cur_score_info['name'])
    cur_crop_inputs = np.zeros((img_count,img_crop_height,img_crop_width,1),dtype='uint8')
    for i in range(img_count):
        img_x = i*img_step
        cur_crop_img = cur_img.crop((img_x, 0, img_x + img_crop_width, img_crop_height))
        cur_crop_inputs[i] = np.array(cur_crop_img).reshape(img_crop_height,img_crop_width,1)
    return cur_crop_inputs 

# FIXME output to file
def output_rmf(outfile,notes_lengths):
    print("writing output to:",outfile)
    with open(outfile,'w') as f:
        cur_beat = 0.0
        #print(notes_lengths)
        for (n,l) in notes_lengths:
            print("note,%d,%f,%f"%(n,cur_beat,l),file=f)
            cur_beat += l


In [5]:
with open(f"data/train2/strip_score_info.json", 'r') as infile:
    score_info = json.load(infile)
    
last_base_name = ""
score_notes_lengths = {}
target_notes_lengths = {}
for cur_score_info in score_info:
    print(cur_score_info['name'])
    cur_base_name = cur_score_info['base']
    if cur_base_name != last_base_name:
        if len(score_notes_lengths.get(last_base_name,[])) > 0:
            rmf_filename = "data/results/%s.rmf"%(last_base_name)
            output_rmf(rmf_filename,score_notes_lengths[last_base_name]) # FIXME
        score_notes_lengths[cur_base_name] = []
        target_notes_lengths[cur_base_name] = get_notes(cur_base_name)
        last_base_name = cur_base_name
    cur_crop_inputs = get_crop_inputs(cur_score_info)
    # predict
    note_preds   = notes_model.predict(cur_crop_inputs)
    length_preds = length_model.predict(cur_crop_inputs)
    raw = get_notes_lengths(note_preds,length_preds)
    nl = refine_notes_lengths(raw)
    #print("x",nl)
    score_notes_lengths[cur_base_name].extend(nl)
rmf_filename = "data/results/%s.rmf"%(last_base_name)
output_rmf(rmf_filename,score_notes_lengths[last_base_name])

strip_score_chromatic_00.png
strip_score_gen_one_00.png
writing output to: data/results/chromatic.rmf
strip_score_gen_one_01.png
strip_score_gen_one_02.png
strip_score_gen_one_03.png
strip_score_gen_one_04.png
strip_score_gen_one_05.png
strip_score_gen_one_06.png
strip_score_gen_one_07.png
strip_score_gen_two_00.png
writing output to: data/results/gen_one.rmf
strip_score_gen_two_01.png
strip_score_gen_two_02.png
strip_score_gen_two_03.png
strip_score_gen_two_04.png
strip_score_gen_two_05.png
strip_score_gen_two_06.png
strip_score_gen_two_07.png
strip_score_gen_two_08.png
strip_score_gen_two_09.png
strip_score_gen_two_10.png
strip_score_gen_two_11.png
strip_score_twinkle_00.png
writing output to: data/results/gen_two.rmf
strip_score_twinkle_01.png
writing output to: data/results/twinkle.rmf
